In [63]:
import gspread
import pandas as pd

SHEET_ID = '1xnOLhct_P9pV2NnWFmuq_apvopvBwkiRjynoFia4iWU'
SHEET_NAME = 'Data_Markets'
gc = gspread.service_account('/Users/aj/Downloads/demo_day/secret_key.json')
spreadsheet = gc.open_by_key(SHEET_ID)
worksheet = spreadsheet.worksheet(SHEET_NAME)
rows = worksheet.get_all_records()
print(rows[:5])

print('==============================')
df = pd.DataFrame(rows)
print(df.head())

[{'OpCo': 'Anomaly', 'National': 'TRUE', 'Regional': '', '': '', 'AL': 1, 'AK': 1, 'AZ': 1, 'AR': 1, 'CA': 1, 'CO': 1, 'CT': 1, 'DC': 1, 'DE': 1, 'FL': 1, 'GA': 1, 'HI': 1, 'ID': 1, 'IL': 1, 'IN': 1, 'IA': 1, 'KS': 1, 'KY': 1, 'LA': 1, 'ME': 1, 'MD': 1, 'MA': 1, 'MI': 1, 'MN': 1, 'MS': 1, 'MO': 1, 'MT': 1, 'NE': 1, 'NV': 1, 'NH': 1, 'NJ': 1, 'NM': 1, 'NY': 1, 'NC': 1, 'ND': 1, 'OH': 1, 'OK': 1, 'OR': 1, 'PA': 1, 'RI': 1, 'SC': 1, 'SD': 1, 'TN': 1, 'TX': 1, 'UT': 1, 'VT': 1, 'VA': 1, 'WA': 1, 'WV': 1, 'WI': 1, 'WY': 1}, {'OpCo': 'Calibrate', 'National': 'TRUE', 'Regional': '', '': '', 'AL': 1, 'AK': 1, 'AZ': 1, 'AR': 1, 'CA': 1, 'CO': 1, 'CT': 1, 'DC': 1, 'DE': 1, 'FL': 1, 'GA': 1, 'HI': 1, 'ID': 1, 'IL': 1, 'IN': 1, 'IA': 1, 'KS': 1, 'KY': 1, 'LA': 1, 'ME': 1, 'MD': 1, 'MA': 1, 'MI': 1, 'MN': 1, 'MS': 1, 'MO': 1, 'MT': 1, 'NE': 1, 'NV': 1, 'NH': 1, 'NJ': 1, 'NM': 1, 'NY': 1, 'NC': 1, 'ND': 1, 'OH': 1, 'OK': 1, 'OR': 1, 'PA': 1, 'RI': 1, 'SC': 1, 'SD': 1, 'TN': 1, 'TX': 1, 'UT': 1, 'VT'

In [64]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scopes = [
    'https://www.googleapis.com/auth/spreadsheets', 
    'https://www.googleapis.com/auth/drive'
]
creds = ServiceAccountCredentials.from_json_keyfile_name("/Users/aj/Downloads/demo_day/secret_key.json", scopes=scopes)

gc = gspread.authorize(creds)
sh = gc.open("Master_Demo_Day_Targets")
sheet_list = sh.worksheets()

In [65]:
sheet_list

[<Worksheet 'Provider_Pipeline' id:77154452>,
 <Worksheet 'Payer_Pipeline' id:1050885711>,
 <Worksheet 'Employer_Pipeline' id:585482382>,
 <Worksheet 'OpCos & Markets' id:70493120>,
 <Worksheet 'InfrastructureMapping ' id:1844632237>,
 <Worksheet 'KPIs' id:1099762335>,
 <Worksheet 'Overview_V2' id:1516551484>,
 <Worksheet 'Data_Markets' id:1857915428>]

In [66]:
sheet2 = sh.worksheet("Data_Markets")

In [67]:
rows = sheet2.get_all_records()
df = pd.DataFrame(rows)
#df
df1 = df.iloc[0:27]

## change weights 
priority, raising rounds in 2023, regional

In [154]:
## edit df1 to reflect weight changes
# this code made to work when priority + raising round 2023 is either/or (not both) -- code may have to change if "both" is a scenario

# would have to edit this (for loop?) to pull from OpCos & Markets tab Priority OpCo column
priority = ["Coral Health", "Fort Health", "Keen Health", "Kins Health", "UpLift"]
raising_round_2023 = ["CardioOne", "Hopewell Health", "Jasper", "Motto Health", "Noma", "Overalls", "Pip (fka Quartz)", "Soundry"]

priority_weight = 20
raising_round_weight = 10
regional = 3

df_priority = pd.DataFrame()
for opco in df1["OpCo"]:
  if opco in priority:
    df_opco = df1[df1["OpCo"] == opco].replace(1, priority_weight)
    df_priority = df_priority.append(df_opco)

df_out = df_priority.combine_first(df1)

df_raising_round = pd.DataFrame()
for opco in df1["OpCo"]:
  if opco in raising_round_2023:
    df_opco = df_out[df_out["OpCo"] == opco].replace(1, raising_round_weight)
    df_raising_round = df_raising_round.append(df_opco)

df_out2 = df_raising_round.combine_first(df_out)

df_regional = pd.DataFrame()
for opco in df1["OpCo"]:
  x = df1[df1["OpCo"] == opco]["Regional"] != ''
  if x.bool() == True:
    df_opco = df_out2[df_out2["OpCo"] == opco].loc[:,'AL':]
    df_opco = df_opco.apply(pd.to_numeric)
    df_opco = df_opco.add(regional)
    df_regional = df_regional.append(df_opco)
  else: 
    df_opco = df_out2[df_out2["OpCo"] == opco]
    df_regional = df_regional.append(df_opco)
df_regional2 = pd.concat([df1.loc[:,:'Regional'], df_regional.loc[:,'AL':]], axis=1)

df2 = df_regional2.fillna(0)

In [155]:
df2

,OpCo,National,Regional,AL,AK,AZ,AR,CA,CO,CT,...,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY
0,Anomaly,TRUE,,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,Calibrate,TRUE,,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,Noma,,"CA, CO, NY, VA",0,0,0,0,13,13,0,...,0,0,0,0,0,0,13,0,0,0
3,CardioOne,,"DE, FL, MD, TX, VA",0,0,0,0,0,0,0,...,0,0,13,0,0,13,0,0,0,0
4,Coral Health,TRUE,,20,20,20,20,20,20,20,...,20,20,20,20,20,20,20,20,20,20
5,DUOS,,TX,0,0,0,0,0,0,0,...,0,0,4,0,0,0,0,0,0,0
6,Fort Health,,"CA, CO, CT, MD, TX, WA",0,0,0,0,23,23,23,...,0,0,23,0,0,0,23,0,0,0
7,Gemini Credentialing,,,,,,,,,,...,,,,,,,,,,
8,Hopewell Health,,"CA, TX",0,0,0,0,13,0,0,...,0,0,13,0,0,0,0,0,0,0
9,Intrinsic,TRUE,,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


## per customer type

In [166]:
### unweighted ###
df2 = df1

### Providers
Providers = Soundry, CardioOne, Primum, Iron Health (fka Aurelia), Hopewell Health, Fort Health, Kins Health, Peppermint, Motiv Health (fka Carmine), Anomaly, Gemini, Pip (fka Quartz), Jasper

In [123]:
providers = ['Soundry', 'CardioOne', 'Primum', 'Iron Health (fka Aurelia)', 'Hopewell Health', 
             'Fort Health', 'Kins Health', 'Peppermint', 'Motiv Health (fka Carmine)', 
             'Anomaly', 'Gemini Credentialing', 'Pip (fka Quartz)', 'Jasper']
new_df = df2[df2["OpCo"].isin(providers)]
#new_df

### Payers
Payers = Motiv Health (fka Carmine), Anomaly, Gemini Credentialing, Pip (fka Quartz), Jasper, MedArrive, Together Senior Health, Noma, Springtide, Keen Health, Uplift, DUOS, Motto Health

In [167]:
payers = ['Motiv Health (fka Carmine)', 'Anomaly', 'Gemini Credentialing', 'Pip (fka Quartz)', 
          'Jasper', 'MedArrive', 'Together Senior Health', 'Noma', 'Springtide', 'Keen Health', 
          'Uplift', 'DUOS', 'Motto Health']

new_df = df2[df2["OpCo"].isin(payers)]

### Employers 
Employers = DUOS, Motto Health, Coral, Overalls, Tempopay, Calibrate, Vault Health 

In [156]:
employers = ['DUOS', 'Motto Health', 'Coral', 'Overalls', 'Tempopay', 'Calibrate', 'Vault Health'] 

new_df = df2[df2["OpCo"].isin(employers)]

## reshape df to fit geospatial viz needs

In [168]:
### for customer type (subset map) ###
df2 = new_df

In [55]:
### for no change ###
#df2 = df1

In [169]:
melted_df = df2.melt(id_vars=['OpCo', 'National', 'Regional'],
        var_name="State", 
        value_name="Value")

In [170]:
melted_df.iloc[0:]

,OpCo,National,Regional,State,Value
0,Anomaly,TRUE,,,
1,Noma,,"CA, CO, NY, VA",,
2,DUOS,,TX,,
3,Gemini Credentialing,,,,
4,Jasper,TRUE,,,
...,...,...,...,...,...
619,Motiv Health (fka Carmine),TRUE,,WY,1
620,Motto Health,,"CA, TX",WY,
621,Pip (fka Quartz),TRUE,,WY,1
622,Springtide,,"CT, MA, NC",WY,


In [171]:
state_df = melted_df.iloc[0:][["State", "Value"]]
state_df_nans = state_df._convert(numeric=True)
state_df1 = state_df_nans.fillna(0)
state_df1

,State,Value
0,,0.0
1,,0.0
2,,0.0
3,,0.0
4,,0.0
...,...,...
619,WY,1.0
620,WY,0.0
621,WY,1.0
622,WY,0.0


In [172]:
df3 = state_df1.groupby("State").sum()
#display(df3) # cross checked with Data_Markets excel sheet + it's correct.
df3.head()

,Value
State,
,0.0
AK,5.0
AL,5.0
AR,5.0
AZ,5.0


In [173]:
states_switched = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC"}

states = {y: x for x, y in states_switched.items()}

In [174]:
df3['State'] = df3.index
df3['State'] = df3['State'].map(lambda x: states.get(x, x))
df3.head()

,Value,State
State,,
,0.0,
AK,5.0,Alaska
AL,5.0,Alabama
AR,5.0,Arkansas
AZ,5.0,Arizona


export as excel 

In [175]:
filename = 'payer_unweighted.xlsx'
df3.to_excel(filename)

## Geospatial viz

In [33]:
!pip install geoplot

  Using cached geoplot-0.5.1-py3-none-any.whl (28 kB)
  Using cached mapclassify-2.4.3-py3-none-any.whl (38 kB)
  Using cached contextily-1.2.0-py3-none-any.whl (16 kB)
  Using cached geopandas-0.12.2-py3-none-any.whl (1.1 MB)
  Using cached xyzservices-2022.9.0-py3-none-any.whl (55 kB)
  Using cached geopy-2.3.0-py3-none-any.whl (119 kB)
  Using cached rasterio-1.3.4-cp39-cp39-macosx_10_15_x86_64.whl (30.9 MB)
  Using cached mercantile-1.2.1-py3-none-any.whl (14 kB)
  Using cached Fiona-1.8.22-cp39-cp39-macosx_10_10_x86_64.whl (26.5 MB)
  Using cached munch-2.5.0-py2.py3-none-any.whl (10 kB)
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached geographiclib-2.0-py3-none-any.whl (40 kB)
  Using cached affine-2.3.1-py2.py3-none-any.whl (16 kB)
  Using cached snuggs-1.4.7-py3-none-any.whl (5.4 kB)


In [18]:
import geoplot as gplt

ImportError: dlopen(/Users/aj/opt/anaconda3/lib/python3.9/site-packages/pyproj/_network.cpython-39-darwin.so, 0x0002): tried: '/Users/aj/opt/anaconda3/lib/python3.9/site-packages/pyproj/_network.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have 'arm64', need 'x86_64')), '/usr/local/lib/_network.cpython-39-darwin.so' (no such file), '/usr/lib/_network.cpython-39-darwin.so' (no such file)

In [17]:
import geoplot as gplt
import geopandas as gpd
import geoplot.crs as gcrs
import imageio
import pandas as pd
import pathlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import mapclassify as mc
import numpy as np

%matplotlib inline

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

ImportError: dlopen(/Users/aj/opt/anaconda3/lib/python3.9/site-packages/pyproj/_network.cpython-39-darwin.so, 0x0002): tried: '/Users/aj/opt/anaconda3/lib/python3.9/site-packages/pyproj/_network.cpython-39-darwin.so' (mach-o file, but is an incompatible architecture (have 'arm64', need 'x86_64')), '/usr/local/lib/_network.cpython-39-darwin.so' (no such file), '/usr/lib/_network.cpython-39-darwin.so' (no such file)

In [ ]:
!wget https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_20m.zip
!mkdir maps
!cd maps
!unzip /content/cb_2018_us_state_20m.zip

In [ ]:
usa = gpd.read_file("maps/cb_2018_us_state_20m.shp")
#state_pop = pd.read_csv("data/nst-est2018-alldata.csv") # don't need state population data...
usa[usa.NAME=="California"].plot()

In [ ]:
### alternatively... we can create a GeoDataFrame (a dataframe with geospatial data) 
# by loading one of the sample datasets from geoplot, in this case the polygones for state boundaries:

gplt_path = gplt.datasets.get_path("contiguous_usa")
contiguous_usa = gpd.read_file(gplt_path)
contiguous_usa.head()
gplt.polyplot(contiguous_usa)

## use RH data for geospatial viz -- 

In [ ]:
# load dataset by state
df3.head()

In [ ]:
# convert into GeoDataFrame using a join. NOTE: this adds the required "geometry" column
geo_df3 = contiguous_usa.set_index("state").join(df3.set_index("State"))
geo_df3.head()

In [ ]:
## weight change: 
ax = gplt.polyplot(contiguous_usa, projection=gcrs.AlbersEqualArea())

gplt.choropleth(
    geo_df3, 
    hue="Value", 
    edgecolor="white", 
    linewidth=1, 
    cmap="Greens", 
    legend=True, 
    scheme="FisherJenks", 
    #legend_labels=[]
    #scale="xxx",
    projection=gcrs.AlbersEqualArea(), 
    ax=ax
)